In [ ]:
# @title 1. 한글 폰트 설치 및 환경 설정 (실행 후 런타임 재시작 필요)
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import pandas as pd
import numpy as np
import seaborn as sns

# 폰트 설정
plt.rc('font', family='NanumBarunGothic')
plt.rcParams['axes.unicode_minus'] = False # 마이너스 기호 깨짐 방지
pd.set_option('display.max_columns', None)

print("환경 설정이 완료되었습니다. 한글 폰트 적용을 위해 '런타임 > 런타임 다시 시작'을 해주세요.")

In [ ]:
# @title 2. 데이터 분석 및 시각화 수행

# --- 파일 경로 설정 (업로드한 파일명과 정확히 일치해야 합니다) ---
file_status = 'KOFOTI_통계보고서_24년-3월25년-2월-패션-소비-실태조사-1.xlsx - 1-2.패션 소비 실태.csv'
file_kids = 'KOFOTI_통계보고서_24년-3월25년-2월-패션-소비-실태조사-1.xlsx - 2-8. 유아동복.csv'
file_casual = 'KOFOTI_통계보고서_24년-3월25년-2월-패션-소비-실태조사-1.xlsx - 2-3. 캐주얼복.csv'

# ==============================================================================
# [분석 1] 연령대별 아동복 구매율 비교 (구매 주체 검증)
# ==============================================================================
print("\n>>> [분석 1] 연령대별 아동복 구매율 비교")

try:
    # 데이터 로드 (헤더가 7번째 줄 근처에 위치함)
    df_status = pd.read_csv(file_status, header=6)

    # '유아동복' 컬럼 찾기
    kids_col = [c for c in df_status.columns if '유아동복' in str(c)][0]

    # '연령' 데이터 추출
    # 첫 번째 컬럼이 '연령'인 행을 찾고, 그 아래 행들을 가져옴
    start_row_idx = df_status[df_status.iloc[:, 0] == '연령'].index[0]
    # 보통 10대~60대까지 6개 구간 (16~19세, 20대, 30대, 40대, 50대, 60대)
    df_age = df_status.iloc[start_idx:start_idx+6].copy()

    # 필요한 컬럼만 선택 (연령대, 유아동복 구매율)
    df_age = df_age.iloc[:, [1, df_status.columns.get_loc(kids_col)]]
    df_age.columns = ['Age_Group', 'Penetration_Rate']
    df_age['Penetration_Rate'] = pd.to_numeric(df_age['Penetration_Rate'], errors='coerce')

    print(df_age)

    # 시각화
    plt.figure(figsize=(10, 6))
    sns.barplot(x='Age_Group', y='Penetration_Rate', data=df_age, palette='Blues')
    plt.title('연령대별 아동복 구매율 (3040세대 핵심 타겟 검증)')
    plt.ylabel('구매율 (%)')
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.show()

except Exception as e:
    print(f"분석 1 오류 발생: {e}")


# ==============================================================================
# [분석 2] 유아동복 vs 캐주얼복 품목별 평균 구매 개수 비교 (거래 주기 검증)
# ==============================================================================
print("\n>>> [분석 2] 유아동복 vs 캐주얼복 품목별 평균 구매 개수 비교")

def extract_purchase_freq(file_path, category_label):
    """
    각 복종 파일에서 품목명과 평균 구매 개수를 추출하는 함수
    """
    try:
        df = pd.read_csv(file_path, header=None)

        # 데이터가 시작되는 행 찾기 (대략 10행 이후)
        # 구조: [ , 품목명, 구매율, 평균구매개수, ...]
        data = []
        start_row = 10

        for i in range(start_row, len(df)):
            item_name = str(df.iloc[i, 1]).strip()

            # 불필요한 행 건너뛰기 (빈 값, 전체 합계 행, 소제목 등)
            if item_name == 'nan' or '전체' in item_name or '성별' in item_name:
                continue

            try:
                # 엑셀 구조상 3번째 컬럼(인덱스 3)이 '평균 구매 개수'로 추정됨
                # (파일마다 다를 수 있으니 값 확인 필요)
                freq = float(str(df.iloc[i, 3]).replace(',', ''))

                # 유효한 숫자인지 확인 (너무 크거나 0이면 제외)
                if freq > 0 and freq < 100:
                    data.append({'Category': category_label, 'Item': item_name, 'Frequency': freq})
            except:
                continue

        return pd.DataFrame(data)
    except Exception as e:
        print(f"{category_label} 데이터 추출 중 오류: {e}")
        return pd.DataFrame()

# 데이터 추출
df_kids_freq = extract_purchase_freq(file_kids, '아동복')
df_casual_freq = extract_purchase_freq(file_casual, '성인 캐주얼')

# 비교를 위해 데이터 합치기
# 품목명이 정확히 일치하지 않을 수 있으므로, 주요 품목(코트, 점퍼, 티셔츠, 바지 등)만 필터링하거나
# 유사한 이름끼리 매칭하는 것이 좋음. 여기서는 전체 리스트를 합쳐서 시각화.
df_comparison = pd.concat([df_kids_freq, df_casual_freq])

# 주요 공통 품목만 필터링 (예시)
common_items = ['코트', '점퍼', '자켓', '티셔츠', '셔츠', '가디건', '바지', '청바지', '스커트', '원피스']
# 품목명에 공통 키워드가 포함된 경우만 남김
df_comparison_filtered = df_comparison[df_comparison['Item'].apply(lambda x: any(k in x for k in common_items))]

print(df_comparison_filtered.head())

# 시각화 (품목별 비교)
plt.figure(figsize=(14, 7))
sns.barplot(x='Item', y='Frequency', hue='Category', data=df_comparison_filtered, palette='Set2')
plt.title('주요 품목별 연간 평균 구매 개수 비교 (아동복 vs 성인 캐주얼)')
plt.ylabel('평균 구매 개수 (개)')
plt.xticks(rotation=45)
plt.legend(title='복종 카테고리')
plt.grid(axis='y', linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()

# 구매 빈도 차이 통계 출력
avg_kids_freq = df_kids_freq['Frequency'].mean()
avg_casual_freq = df_casual_freq['Frequency'].mean()
print(f"\n[결과 요약]")
print(f"- 아동복 전체 평균 구매 빈도: {avg_kids_freq:.2f}개")
print(f"- 성인 캐주얼 전체 평균 구매 빈도: {avg_casual_freq:.2f}개")
print(f"- 아동복이 성인복보다 평균적으로 약 {(avg_kids_freq/avg_casual_freq - 1)*100:.1f}% 더 자주 구매됨")